CODE_FOR_PEOPLE_EYE_AND_YAWN_DETECTION

In [1]:
import cv2
import dlib
import time

# Load face detector and facial landmark predictor
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Function to calculate eye aspect ratio
def eye_aspect_ratio(eye_points):
    # Vertical distances
    vertical_1 = ((eye_points[1].x - eye_points[5].x) ** 2 + (eye_points[1].y - eye_points[5].y) ** 2) ** 0.5
    vertical_2 = ((eye_points[2].x - eye_points[4].x) ** 2 + (eye_points[2].y - eye_points[4].y) ** 2) ** 0.5

    # Horizontal distance
    horizontal = ((eye_points[0].x - eye_points[3].x) ** 2 + (eye_points[0].y - eye_points[3].y) ** 2) ** 0.5

    # Eye aspect ratio
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Function to detect closed eyes
def detect_closed_eyes(frame, landmarks):
    left_eye_points = [landmarks.part(i) for i in range(36, 42)]
    right_eye_points = [landmarks.part(i) for i in range(42, 48)]

    left_ear = eye_aspect_ratio(left_eye_points)
    right_ear = eye_aspect_ratio(right_eye_points)

    # Average eye aspect ratio
    ear = (left_ear + right_ear) / 2.0

    # Draw dots and lines around the eyes
    for i in range(36, 48):
        cv2.circle(frame, (landmarks.part(i).x, landmarks.part(i).y), 2, (255, 255, 255), -1)

    return ear

# Function to detect yawning
def detect_yawn(frame, landmarks):
    # Get the mouth region from landmarks
    mouth_points = [landmarks.part(i) for i in range(48, 68)]
    
    if len(mouth_points) == 20:
        # Calculate the aspect ratio of the mouth
        mouth_width = abs(mouth_points[6].x - mouth_points[0].x)
        mouth_height = abs(mouth_points[3].y - mouth_points[9].y)
        aspect_ratio = mouth_height / mouth_width
        
        # Draw dots and lines around the mouth
        for i in range(20):
            cv2.circle(frame, (mouth_points[i].x, mouth_points[i].y), 2, (255, 255, 255), 2)
            cv2.line(frame, (mouth_points[i].x, mouth_points[i].y), (mouth_points[(i+1)%20].x, mouth_points[(i+1)%20].y), (255, 255, 255), 2)
        
        return aspect_ratio

    return None

# Open video capture (replace 'path_to_video' with 0 for webcam)
cap = cv2.VideoCapture("People_Yawning.mp4")

# Set the window size
cv2.namedWindow('Drowsiness and Yawn Detection', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Drowsiness and Yawn Detection', 800, 700)  # Adjust the width and height as desired

# Initialize variables for eye detection and score
closed_eye_start = None
closed_eye_alert_threshold = 0.5  # Number of seconds with closed eyes required to show the alert

eye_closed_count = 0  # Counter variable for eye closed detections
yawn_count = 0  # Counter variable for yawn detections

# Process video frames until the user exits
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using dlib's face detector
    faces = face_detector(gray)

    for face in faces:
        # Detect landmarks in the face region
        landmarks = landmark_predictor(gray, face)

        # Detect closed eyes
        ear = detect_closed_eyes(frame, landmarks)

        # Check if eyes are closed
        ear_threshold = 0.25  # Adjust this value as desired
        if ear < ear_threshold:
            if closed_eye_start is None:
                closed_eye_start = time.time()
            else:
                closed_eye_duration = time.time() - closed_eye_start

                # Check if the closed eye duration exceeds the alert threshold
                if closed_eye_duration >= closed_eye_alert_threshold:
                    cv2.putText(frame, "Drowsiness Detected", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    eye_closed_count += 1  # Increment eye closed count

        else:
            closed_eye_start = None

        # Detect yawning
        aspect_ratio = detect_yawn(frame, landmarks)
        if aspect_ratio is not None:
            yawn_threshold = 1.5  # Adjust this value as desired
            if aspect_ratio > yawn_threshold:
                cv2.putText(frame, "Yawn Detected", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                yawn_count += 1  # Increment detection count

    cv2.imshow('Drowsiness and Yawn Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()
